In [1]:
%%capture
%pip install -U bitsandbytes
%pip install transformers==4.44.2
%pip install -U accelerate
%pip install -U peft
%pip install -U trl

In [2]:
from huggingface_hub import login
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()

hf_token = user_secrets.get_secret("HUGGINGFACE_TOKEN")
login(token = hf_token)

In [3]:
base_model_url = "/kaggle/input/gemma/transformers/2b-it/3"
new_model_url = "/kaggle/input/gemma-finetune-test/gemma-2b-it-v3-wealth-finetune/"

In [4]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, pipeline
from peft import PeftModel
import torch
from trl import setup_chat_format

tokenizer = AutoTokenizer.from_pretrained(base_model_url)

base_model_reload= AutoModelForCausalLM.from_pretrained(
    base_model_url,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map="auto",
)

2025-05-15 02:00:06.756758: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747274406.948776      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747274407.008022      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [5]:
tokenizer.chat_template = None
base_model_reload, tokenizer = setup_chat_format(base_model_reload, tokenizer)
model = PeftModel.from_pretrained(base_model_reload, new_model_url)

model = model.merge_and_unload()

The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


In [6]:
instruction = """You are a wealth management assistant. 
     Be professional be matter of fact about your points.
    """

messages = [{"role": "system", "content": instruction},
    {"role": "user", "content": "What are common investing strategies?"}]

prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    
inputs = tokenizer(prompt, return_tensors='pt', padding=True, truncation=True).to("cuda")

outputs = model.generate(**inputs, max_new_tokens=150, num_return_sequences=1)

text = tokenizer.decode(outputs[0], skip_special_tokens=True)

prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

inputs = tokenizer(prompt, return_tensors='pt').to("cuda")

outputs = model.generate(**inputs, max_new_tokens=150)

generated_tokens = outputs[0][inputs['input_ids'].shape[-1]:]

response = tokenizer.decode(generated_tokens, skip_special_tokens=True)

print(response.strip())

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Sure, I can help you with that! Investing strategies are a set of guidelines or rules that investors use to make informed decisions about their investments. Here are some common investing strategies:

1. Diversification: Diversification involves spreading your investments across different asset classes, such as stocks, bonds, and real estate. This strategy helps reduce risk and maximize potential returns.

2. Value Investing: Value investing focuses on finding undervalued stocks or assets that have the potential to grow in value. Investors use metrics like price-to-earnings (P/E) ratios and dividend yields to identify these stocks.

3. Growth Investing: Growth investing involves investing in stocks or assets with high growth potential. Investors use metrics like revenue growth, earnings per share (


In [7]:
new_model = "gemma-2b-it-v3-wealth-finetune-merged"

model.save_pretrained(new_model)
tokenizer.save_pretrained(new_model)

('gemma-2b-it-v3-wealth-finetune-merged/tokenizer_config.json',
 'gemma-2b-it-v3-wealth-finetune-merged/special_tokens_map.json',
 'gemma-2b-it-v3-wealth-finetune-merged/tokenizer.model',
 'gemma-2b-it-v3-wealth-finetune-merged/added_tokens.json',
 'gemma-2b-it-v3-wealth-finetune-merged/tokenizer.json')

In [8]:
model.push_to_hub(new_model, use_temp_dir=False)
tokenizer.push_to_hub(new_model, use_temp_dir=False)

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

No files have been modified since last commit. Skipping to prevent empty commit.
No files have been modified since last commit. Skipping to prevent empty commit.


CommitInfo(commit_url='https://huggingface.co/young43/gemma-2b-it-v3-wealth-finetune-merged/commit/b22f207dba7ece7e238d9b0abe894c9caccb880d', commit_message='Upload tokenizer', commit_description='', oid='b22f207dba7ece7e238d9b0abe894c9caccb880d', pr_url=None, repo_url=RepoUrl('https://huggingface.co/young43/gemma-2b-it-v3-wealth-finetune-merged', endpoint='https://huggingface.co', repo_type='model', repo_id='young43/gemma-2b-it-v3-wealth-finetune-merged'), pr_revision=None, pr_num=None)